Anzeige der Temperaturdaten aus der Heizung

In [121]:
import datetime
import sqlite3
import numpy as np
import pandas as pd
import os, sys
import ipywidgets as widgets
import matplotlib.pyplot as pl
DB_FILENAME = 'Temperaturen.db'


def get_df(db_filename):
    db_file = db_filename
    
    if os.path.exists(os.path.join('/var/lib/grafana', db_filename)):
        db_file = os.path.join('/var/lib/grafana', db_filename)
    elif os.path.exists(os.path.join('.', db_filename)):
        db_file = os.path.join('.', db_filename)    
    else:
        raise SystemExit('Die Datenbank ' + db_file + ' ist nicht vorhanden')
        
    try:
        con = sqlite3.connect(db_file)
        df = pd.read_sql_query("SELECT * from Temperaturen", con)
    except:
        raise SystemExit('Die Datenbank ' + db_file + ' ist nicht vorhanden')
    # Blanks mit np.nan ersetzen
    return df.replace(r'^\s*$', np.nan, regex=True)
        



In [135]:
def plot_temp(start, dauer):

    print('Start: ' + str(start))
    print(dauer)
    start_object = datetime.datetime.strptime(start, '%Y-%m-%d')
    end_date = start_object + datetime.timedelta(days=dauer)
    print('Ende: ' + str(end_date))
    # Hier nach Start und Enddatum filtern
    subset = df[(df['UnixTime'] > start ) &  (df['UnixTime'] < end_date )]
    #df = df['date'] + pd.DateOffset(months=plus_month_period)
    #data.info()
    # UnixTime 	AussenTemp 	VorlaufTemp 	RuecklaufTemp 	WohnzimmerTemp
    subset.plot(x="UnixTime", y=["VorlaufTemp", "RuecklaufTemp", "WohnzimmerTemp", "AussenTemp"])
    #data    


# main
try:    
    df= get_df(DB_FILENAME)
except SystemExit as _e:
    print(_e)
    sys.exit(-1)
    
df['UnixTime'] = pd.to_datetime(df['UnixTime'], unit='s')


dates_us = df.UnixTime.dt.strftime('%Y-%m-%d').unique()


df.set_index(['UnixTime'])
df['Aussen'] = str(df['AussenTemp'])
df['Wohnzimmer'] = str(df['WohnzimmerTemp'])

widget_dauer = widgets.IntSlider(start=1,end=20,step=1, description="Dauer", value=2)
widget_start = widgets.Select(
    options=np.sort(dates_us)[::-1],
    rows=1,
    description='Startdatum:',
    disabled=False
)

widgets.interact(plot_temp, start= widget_start, dauer = widget_dauer)

interactive(children=(Select(description='Startdatum:', options=('2023-10-17', '2023-10-16', '2023-05-19', '20…

<function __main__.plot_temp(start, dauer)>